# Chapter 4: Training Models

Pada chapter ini, pembahasan Machine Learning berfokus pada **bagaimana sebuah model dilatih secara matematis dan algoritmik**. Jika pada chapter sebelumnya kita banyak menggunakan model sebagai *black box*, maka chapter ini mengajak pembaca untuk melihat apa yang sebenarnya terjadi *di balik layar*.

Pemahaman mengenai proses training sangat penting karena membantu kita:
- memilih model yang tepat untuk suatu permasalahan,
- menentukan algoritma optimisasi yang sesuai,
- melakukan debugging ketika model tidak bekerja dengan baik,
- serta memahami konsep yang menjadi dasar neural networks.

Chapter ini membahas beberapa topik utama, antara lain:
- Linear Regression dan cara melatihnya,
- Gradient Descent dan variasinya,
- Polynomial Regression,
- bias–variance trade-off,
- regularization,
- hingga Logistic Regression dan Softmax Regression.

## 1. Linear Regression

Linear Regression merupakan salah satu model Machine Learning paling sederhana dan paling fundamental. Model ini mengasumsikan bahwa hubungan antara fitur input dan target dapat direpresentasikan sebagai **kombinasi linear**.

Secara intuitif, Linear Regression mencoba mencari garis (atau bidang, atau hiperbidang) yang paling sesuai dengan data, sehingga kesalahan prediksi terhadap data training menjadi sekecil mungkin.

### 1.1 Model Linear Regression

Untuk sebuah instance dengan beberapa fitur, Linear Regression memprediksi nilai target dengan menghitung **jumlah tertimbang dari seluruh fitur**, kemudian menambahkan sebuah konstanta yang disebut *bias* atau *intercept*.

Secara matematis, model Linear Regression dapat dituliskan sebagai:

\[ \hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n \]

Di mana:
- \( \hat{y} \) adalah nilai prediksi,
- \( x_i \) adalah nilai fitur ke-i,
- \( \theta_i \) adalah parameter model yang dipelajari dari data.

Model ini juga dapat dituliskan dalam bentuk vektorisasi untuk memudahkan komputasi.

## 2. Generating Linear Data

Untuk memahami bagaimana Linear Regression bekerja dan bagaimana model dilatih, kita akan membuat dataset sintetis sederhana yang mengikuti hubungan linear, kemudian menambahkan noise.

Dataset ini akan digunakan untuk mendemonstrasikan berbagai metode training pada Linear Regression.

In [ ]:
import numpy as np

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

Data di atas dibuat menggunakan fungsi linear \( y = 4 + 3x \) dengan tambahan *Gaussian noise*. Noise ini mencerminkan kondisi dunia nyata, di mana data jarang bersifat sempurna.

## 3. Training Linear Regression Using the Normal Equation

Salah satu cara untuk melatih model Linear Regression adalah dengan menggunakan **Normal Equation**. Pendekatan ini mencari solusi optimal secara langsung melalui perhitungan matematis, tanpa memerlukan iterasi bertahap.

Normal Equation memberikan solusi parameter \( \theta \) yang meminimalkan *Mean Squared Error* secara analitik.

### 3.1 Normal Equation

Secara matematis, solusi Linear Regression menggunakan Normal Equation dapat dituliskan sebagai:

\[ \theta = (X^T X)^{-1} X^T y \]

Rumus ini menghitung parameter optimal dalam satu langkah. Namun, pendekatan ini memiliki keterbatasan karena kompleksitas komputasinya meningkat drastis ketika jumlah fitur sangat besar.

In [ ]:
X_b = np.c_[np.ones((100, 1)), X]
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
theta_best

Hasil di atas menunjukkan nilai parameter \( \theta_0 \) dan \( \theta_1 \) yang mendekati nilai asli yang digunakan untuk menghasilkan data, yaitu 4 dan 3.

Perbedaan kecil terjadi karena adanya noise pada data.

### 3.2 Making Predictions

Setelah parameter model diperoleh, kita dapat menggunakan model Linear Regression untuk melakukan prediksi pada data baru.

In [ ]:
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]
y_predict = X_new_b.dot(theta_best)
y_predict

Prediksi di atas menunjukkan nilai target yang diperkirakan oleh model untuk input baru. Model menggunakan parameter yang telah dipelajari untuk melakukan generalisasi.

## 4. Linear Regression Using Scikit-Learn

Pada praktiknya, kita jarang menghitung Normal Equation secara manual. Library seperti **Scikit-Learn** menyediakan implementasi Linear Regression yang efisien dan stabil secara numerik.

Implementasi ini secara internal menggunakan teknik optimisasi yang setara, tanpa mengharuskan kita menghitung invers matriks secara eksplisit.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

Nilai intercept dan koefisien yang dihasilkan oleh Scikit-Learn sangat mendekati hasil dari Normal Equation, membuktikan bahwa kedua pendekatan menghasilkan solusi yang sama.

### 4.1 Predictions Using Scikit-Learn

Model Linear Regression dari Scikit-Learn dapat langsung digunakan untuk melakukan prediksi tanpa perlu menambahkan bias secara manual.

In [ ]:
lin_reg.predict(X_new)

Hasil prediksi konsisten dengan prediksi sebelumnya, menunjukkan bahwa Scikit-Learn menangani detail implementasi secara otomatis.

## 5. Gradient Descent

Ketika jumlah fitur sangat besar, pendekatan Normal Equation menjadi tidak efisien karena membutuhkan invers matriks. Untuk kondisi ini, digunakan metode optimisasi iteratif yang disebut **Gradient Descent**.

Gradient Descent bekerja dengan menyesuaikan parameter model secara bertahap, bergerak ke arah yang menurunkan nilai fungsi biaya (*cost function*) hingga mencapai minimum.

### 5.1 The Cost Function

Untuk Linear Regression, fungsi biaya yang umum digunakan adalah **Mean Squared Error (MSE)**. Fungsi ini mengukur rata-rata kuadrat selisih antara nilai prediksi dan nilai aktual.

Gradient Descent bertujuan meminimalkan nilai MSE dengan memperbarui parameter \( \theta \) secara iteratif.

### 5.2 Batch Gradient Descent

Pada **Batch Gradient Descent**, gradien dihitung menggunakan seluruh data training pada setiap iterasi. Pendekatan ini memberikan arah gradien yang stabil, namun bisa menjadi lambat untuk dataset yang besar.

In [ ]:
eta = 0.1
n_iterations = 1000
m = 100

theta = np.random.randn(2, 1)

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

theta

Hasil parameter yang diperoleh melalui Batch Gradient Descent mendekati hasil Normal Equation. Hal ini menunjukkan bahwa metode iteratif mampu mencapai solusi optimal jika *learning rate* dipilih dengan tepat.

### 5.3 Learning Rate

*Learning rate* (\( \eta \)) menentukan seberapa besar langkah pembaruan parameter pada setiap iterasi.

- Learning rate terlalu kecil menyebabkan konvergensi lambat
- Learning rate terlalu besar dapat menyebabkan divergensi

Pemilihan learning rate yang tepat sangat krusial dalam proses training model.

Secara konseptual, perilaku Gradient Descent dengan berbagai nilai learning rate sering divisualisasikan untuk menunjukkan perbedaan antara konvergensi yang stabil dan divergensi, seperti yang digambarkan pada ilustrasi di buku.

## 6. Stochastic Gradient Descent (SGD)

Berbeda dengan Batch Gradient Descent yang menggunakan seluruh data training pada setiap iterasi, **Stochastic Gradient Descent (SGD)** memperbarui parameter model menggunakan **satu instance data secara acak** pada setiap langkah.

Pendekatan ini membuat proses training menjadi jauh lebih cepat, terutama pada dataset yang sangat besar. Namun, karena arah gradien menjadi lebih berisik (*noisy*), proses konvergensinya tidak selalu stabil.

### 6.1 Karakteristik SGD

Beberapa karakteristik utama dari Stochastic Gradient Descent adalah:
- Pembaruan parameter sangat cepat
- Cocok untuk dataset berskala besar atau *online learning*
- Jalur menuju minimum fungsi biaya berosilasi
- Sulit mencapai minimum absolut, tetapi dapat mendekati minimum global

Osilasi ini justru membantu SGD keluar dari minimum lokal yang buruk.

### 6.2 Learning Schedule

Untuk mengatasi osilasi pada SGD, sering digunakan **learning schedule**, yaitu strategi menurunkan learning rate secara bertahap selama training.

Dengan learning rate yang semakin kecil, langkah pembaruan parameter menjadi lebih halus ketika model mendekati solusi optimal.

In [ ]:
n_epochs = 50
t0, t1 = 5, 50

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2, 1)

for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients

theta

Hasil parameter dari SGD mendekati solusi optimal, meskipun jalur konvergensinya tidak sehalus Batch Gradient Descent.

## 7. Mini-batch Gradient Descent

**Mini-batch Gradient Descent** merupakan kompromi antara Batch Gradient Descent dan Stochastic Gradient Descent.

Pada pendekatan ini, gradien dihitung menggunakan **subset kecil (mini-batch)** dari data training pada setiap iterasi.

### 7.1 Keunggulan Mini-batch Gradient Descent

Mini-batch Gradient Descent memiliki beberapa keunggulan:
- Lebih stabil dibandingkan SGD
- Lebih cepat dibandingkan Batch Gradient Descent
- Efisien pada perangkat keras modern (CPU/GPU)

Karena alasan inilah, mini-batch Gradient Descent menjadi pendekatan standar dalam training neural networks.

Secara konseptual, perbedaan antara Batch, Stochastic, dan Mini-batch Gradient Descent sering divisualisasikan untuk menunjukkan perbedaan jalur optimisasi menuju minimum fungsi biaya, sebagaimana digambarkan pada ilustrasi di buku.

## 8. Polynomial Regression

Linear Regression memiliki keterbatasan ketika hubungan antara fitur dan target bersifat non-linear. Untuk mengatasi hal ini, kita dapat menggunakan **Polynomial Regression**, yaitu dengan menambahkan fitur-fitur polinomial dari fitur asli.

Meskipun namanya Polynomial Regression, model ini tetap merupakan model linear terhadap parameter, karena hubungan linear terjadi pada koefisien \( \theta \), bukan pada fitur.

### 8.1 Generating Nonlinear Data

Untuk mendemonstrasikan Polynomial Regression, kita membuat dataset sintetis dengan hubungan non-linear antara fitur dan target.

In [ ]:
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

Data di atas mengikuti hubungan kuadratik dengan tambahan noise, sehingga tidak dapat dimodelkan dengan baik menggunakan Linear Regression biasa.

### 8.2 Polynomial Features

Scikit-Learn menyediakan transformer `PolynomialFeatures` untuk menambahkan fitur polinomial secara otomatis.

Pada contoh ini, kita menambahkan fitur hingga derajat 2 (kuadratik).

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)
X_poly[0]

Setiap instance kini memiliki fitur tambahan berupa kuadrat dari fitur asli, memungkinkan model menangkap hubungan non-linear.

### 8.3 Polynomial Regression Model

Setelah fitur polinomial dibuat, kita dapat melatih model Linear Regression seperti biasa.

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.coef_, lin_reg.intercept_

Model ini kini mampu memodelkan hubungan non-linear pada data, karena fitur-fitur polinomial telah disertakan.

## 9. Underfitting and Overfitting

**Underfitting** terjadi ketika model terlalu sederhana untuk menangkap pola dalam data, sedangkan **overfitting** terjadi ketika model terlalu kompleks dan mulai mempelajari noise.

Polynomial Regression dengan derajat rendah cenderung underfitting, sementara derajat tinggi cenderung overfitting.

## 10. Learning Curves

Untuk menganalisis apakah model mengalami underfitting atau overfitting, kita dapat menggunakan **learning curves**.

Learning curve memplot performa model pada training set dan validation set sebagai fungsi dari jumlah data training.

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, val_scores = learning_curve(
    LinearRegression(), X, y,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring="neg_mean_squared_error"
)

train_rmse = np.sqrt(-train_scores.mean(axis=1))
val_rmse = np.sqrt(-val_scores.mean(axis=1))

train_rmse, val_rmse

Learning curves membantu kita mengidentifikasi apakah menambah data training atau meningkatkan kompleksitas model dapat memperbaiki performa.

Jika kedua kurva mendekat namun error masih tinggi, model cenderung underfitting. Jika terdapat jarak besar antara kurva training dan validation, model cenderung overfitting.

## 11. Regularized Linear Models

Ketika model terlalu kompleks, risiko **overfitting** meningkat. Salah satu teknik utama untuk mengatasi masalah ini adalah **regularization**, yaitu menambahkan penalti pada besar parameter model.

Regularization mendorong model untuk memilih parameter yang lebih kecil, sehingga model menjadi lebih sederhana dan lebih mampu melakukan generalisasi.

### 11.1 Ridge Regression (L2 Regularization)

Ridge Regression menambahkan penalti berupa kuadrat dari parameter ke dalam fungsi biaya. Pendekatan ini mengecilkan nilai parameter tanpa menghilangkannya sepenuhnya.

Ridge Regression sangat efektif ketika semua fitur memiliki kontribusi kecil terhadap prediksi.

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=1)
ridge_reg.fit(X, y)
ridge_reg.coef_, ridge_reg.intercept_

Parameter `alpha` mengontrol kekuatan regularization. Semakin besar nilai alpha, semakin kuat penalti yang diberikan pada parameter model.

### 11.2 Lasso Regression (L1 Regularization)

Lasso Regression menggunakan penalti absolut dari parameter. Salah satu sifat unik Lasso adalah kemampuannya untuk membuat beberapa parameter bernilai nol, sehingga berperan sebagai **feature selection**.

Pendekatan ini sangat berguna ketika hanya sebagian kecil fitur yang benar-benar relevan.

In [ ]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)
lasso_reg.coef_, lasso_reg.intercept_

Koefisien bernilai nol menunjukkan fitur yang dianggap tidak penting oleh model.

### 11.3 Elastic Net

Elastic Net menggabungkan regularization L1 dan L2. Pendekatan ini berguna ketika terdapat banyak fitur yang saling berkorelasi.

Elastic Net menghindari kelemahan Lasso yang cenderung memilih satu fitur dari kelompok fitur yang berkorelasi.

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X, y)
elastic_net.coef_, elastic_net.intercept_

## 12. Logistic Regression

Meskipun namanya mengandung kata *regression*, **Logistic Regression** digunakan untuk **classification**, terutama binary classification.

Model ini memprediksi probabilitas suatu instance termasuk ke dalam kelas tertentu menggunakan fungsi sigmoid.

### 12.1 Logistic Function

Fungsi sigmoid memetakan nilai input ke rentang 0 hingga 1, sehingga cocok untuk merepresentasikan probabilitas.

Berdasarkan probabilitas ini, model mengambil keputusan kelas menggunakan threshold tertentu.

## 13. Softmax Regression

Softmax Regression merupakan generalisasi Logistic Regression untuk **multiclass classification**.

Alih-alih memprediksi probabilitas satu kelas, Softmax memprediksi probabilitas untuk setiap kelas, dan memilih kelas dengan probabilitas tertinggi.

## Closing Summary (Chapter 4)

Chapter 4 membahas secara mendalam bagaimana model Machine Learning dilatih, mulai dari pendekatan matematis hingga algoritma optimisasi yang digunakan dalam praktik.

Melalui Linear Regression, Gradient Descent, Polynomial Regression, regularization, dan Logistic Regression, kita memahami bahwa proses training adalah inti dari Machine Learning.

Pemahaman konsep-konsep ini menjadi fondasi penting untuk mempelajari model yang lebih kompleks seperti Support Vector Machines dan Neural Networks pada chapter berikutnya.